# Vector Search with Cloud SQL

Please adapt the password and hostname entries in the following cell to match your Cloud SQL deployment.

In [ ]:
username = 'SQLAdmin'
password = '...'
hostname = '...'
port = 443 
namespace = 'USER'

## Setting up the SSL configuration

SSL configurations for DB-API connections are based on the same infrastructure as ODBC and .NET. That is, they'll look for configuration settings in a `.ini` file specified through the `ISC_SSLconfigurations` environment variable. The following code snippet will check if the file exists and, if it doesn't, set one up for you based on sane defaults for our simple connection scenario.

In [ ]:
import iris, os

# The SSL config name we'll use when connecting
sslConfigName = "CloudSQL"

# Check if the ISC_SSLconfigurations environment variable is set
sslConfigFile = os.getenv('ISC_SSLconfigurations')
if (sslConfigFile == None):
    print("ERROR: ISC_SSLconfigurations is not set!")

# if SSL config file does not exist, we can create it
if (os.path.exists(sslConfigFile)):
    print("Found existing SSL config file, assume it's intentional")
else:
    certificateFile = (os.path.dirname(sslConfigFile) + "/certificateSQLaaS.pem")
    ini = "[%s] \n" % sslConfigName
    ini += "CertFile=%s \n" % certificateFile
    ini += "KeyType=2 \n"
    ini += "Protocols=28 \n"
    ini += "CipherList=ALL:!aNULL:!eNULL:!EXP:!SSLv2 \n"
    ini += "VerifyPeer=0 \n"
    ini += "VerifyDepth=9 \n"
    with open(sslConfigFile, 'wt') as f:
        f.write(ini)
    print("Wrote new SSLConfigs.ini file")

    if (os.path.exists(certificateFile) == False):
        print("Please make sure your certificate file is located at '%s' before proceeding" % certificateFile)

### Copying the certificate

Now that the SSL configurations are in place, make sure the `certificateSQLaaS.pem` file for your Cloud SQL deployment is located at the path described in the previous cell's output (defaults to `/usr/cert-demo/`). You can download the certificate file from your deployment's detail screen. Look for the button that says "Get X.509 certificate".

Next, you'll have to copy it into the container. Adapt the download location of the certificate and the name of the target container in following command to suit your environment.

```Shell
docker cp ~/Downloads/certificateSQLaas.pem iris-vector-search-jupyter-1:/usr/cert-demo/
```

After this finishes, all we need to do is make sure DB-API will pick up the SSL configuration we configured in the `.ini` file, referencing it by name in the optional `sslconfig` argument for `iris.connect()`:

In [ ]:
c = iris.connect(hostname, port, namespace, username, password, sslconfig = sslConfigName )

From now on, this is no different from a regular SQL connection, so we can just move ahead!